In [1]:
!pip install streamlit --quiet
!npm install localtunnel --quiet
!conda install -y -c conda-forge poppler

m##################) ⠹ reify:yargs: http fetch GET 200 https://registry.npmjs.oistry.
added 22 packages in 1s

3 packages are looking for funding
  run `npm fund` for details
npm notice 
npm notice New major version of npm available! 9.5.0 -> 11.0.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.0.0
npm notice Run npm install -g npm@11.0.0 to update!
npm notice 
Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.3
  latest version: 24.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.11.1



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - poppler


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificat

In [2]:
!pip install pdf2image

In [3]:
%%writefile app.py

import os
import streamlit as st
from pdf2image import convert_from_path
import pytesseract
import torch
from transformers import T5Tokenizer, AdamW, T5ForConditionalGeneration
import torch.nn as nn
from pdf2image import convert_from_bytes


def convert_pdf_to_images(pdf_bytes):
    with open("temp_pdf.pdf", "wb") as f:
        f.write(pdf_bytes)
    try:
        images = convert_from_path("temp_pdf.pdf")
    except Exception as e:
        st.error(f"Failed to convert PDF to images: {e}")
        images = []
    os.remove("temp_pdf.pdf")
    return images

class Process:
  def __init__(self, article_text):
    self.article_text = article_text
    self.art_max_len = Config.ART_MAX_LEN
    self.tokenizer = Config.TOKENIZER


  def pre_process(self):
    article_text = str(self.article_text)
    article_text = " ".join(article_text.split())

    inputs = self.tokenizer.batch_encode_plus(
        [article_text],
        max_length=self.art_max_len,
        pad_to_max_length=True,
        truncation=True,
        padding="max_length"
        )
    
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    return (
        torch.tensor(input_ids, dtype=torch.long),
        torch.tensor(attention_mask, dtype=torch.long)
        )

  def post_process(self, generated_ids):
    preds = [
        self.tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
        ]
    return " ".join(preds)


class Config:
    EPOCHS = 4
    MAX_LEN = 512
    HIG_MAX_LEN = 64
    ART_MAX_LEN = 512
    LEARNING_RATE = 3e-5
    TRAIN_BATCH_SIZE = 4
    VALID_BATCH_SIZE = 4
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    CHECKPOINT = "t5-base"
    MODEL_PATH = "/kaggle/input/t5-transformer/pytorch/default/1/t5trained.bin"

    TOKENIZER = T5Tokenizer.from_pretrained(CHECKPOINT)
    MODEL = T5ForConditionalGeneration.from_pretrained(CHECKPOINT, return_dict=True)
    
    
class T5Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = Config.MODEL

  def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
    output = self.model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )
    return output.loss, output.logits


def main():
    st.title("Nepal Policy Document Summarizer")

    # Create a file uploader that accepts PDF files
    uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")

    if uploaded_file is not None:
        # Display the file name
        st.write("Uploaded file:", uploaded_file.name)
        bytes_data = uploaded_file.read()
        st.write("File size (in bytes):", len(bytes_data))

        # Example: Save the uploaded file locally
        file_path = os.path.join(os.getcwd(), uploaded_file.name)
        with open(file_path, "wb") as f:
            f.write(bytes_data)
            
        images = convert_from_bytes(bytes_data)

        st.write("Performing OCR...")
        all_text = []
        for image in images:
            text = pytesseract.image_to_string(image, lang="eng")
            all_text.append(text)
        
        all_text = [text.replace("\n", "") for text in all_text]
        st.write("OCR Completed...")
        model = CNNDailyMailModel()
        model.to(Config.DEVICE)
        model.load_state_dict(
            torch.load(
                Config.MODEL_PATH,
                map_location=torch.device(Config.DEVICE)
                )
            )
        imp = []
        st.write("Summarizing text....")
        for text in all_text:
            data = Process(text)
            input_ids, attention_mask = data.pre_process()
            input_ids = input_ids.to(Config.DEVICE)
            attention_mask = attention_mask.to(Config.DEVICE)

            with torch.no_grad():
                generated_ids = model.model.generate(
                    input_ids = input_ids,
                    attention_mask = attention_mask,
                    max_length=150,
                    num_beams=2,
                    repetition_penalty=2.5,
                    length_penalty=1.0,
                    early_stopping=True
                    )

            predicted_high = data.post_process(generated_ids)
            imp.append(predicted_high)
        list_as_string = ', '.join(map(str, imp))
        st.write("Summarized text...")
        st.text_area(list_as_string)
            

if __name__ == "__main__":
    main()


Writing app.py


In [4]:
# Your public ip is the password to the localtunnel
!curl ipv4.icanhazip.com

curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
34.91.4.130


In [5]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

your url is: https://green-trains-sneeze.loca.lt
^C


## Test on pdf file, by converting it to image

In [6]:
import pytesseract
import pdf2image
from pdf2image import convert_from_path
import logging
import os
from tqdm import tqdm

In [7]:
!ls

'1657876777323National Health Policy-1991.pdf'	    node_modules
 NEC_Temporary_Certificate2024-10-08_14_09_26.pdf   package-lock.json
 app.py						    package.json
 logs.txt


In [9]:
english_pdf_path ="/kaggle/working/1657876777323National Health Policy-1991.pdf"
def convert_pdf_to_images(pdf_path: str):
    images = convert_from_path(pdf_path)
    return images

english_images = convert_pdf_to_images(english_pdf_path)


In [10]:
english_images

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>]

In [11]:
all_text = []
for image in tqdm(english_images):
    text = pytesseract.image_to_string(image, lang="eng")
    all_text.append(text)

100%|██████████| 5/5 [01:40<00:00, 20.11s/it]


In [12]:
all_text

["NATIONAL HEALTH POLICY, 1991\n\nPRESENT HEALTH STATUS\n\nThe present low level of health status is attributable to lack of political commitment,\ninappropriate strategies and weakness in implementation of health programmes up to the grass\nroot level during the past 30 years. Because of those weaknesses even now the crude death rate is\n16 per thousand, crude birth rate is 41 per thousand, child mortality rate is 107 per thousand,\nmaternal mortality rate is 8.5 per thousand and mortality rate of children below5 years is 197 per\nthousand. These facts and figures have confirmed Nepal as an underdeveloped and backward\nnation.\n\nRegarding health services delivery, there is one hospital for 168 thousand persons and one\ndoctor for 92 thousand persons in the rural areas. Likewise, only one hospital bed is available for\nnearly 4 thousand persons. There is only one Health post for 24 thousand rural persons, which\nindicates the inadequacy of PHC services at the rural level.\n\nMAIN DEFI

In [13]:
all_text = [text.replace("\n", "") for text in all_text]
all_text

["NATIONAL HEALTH POLICY, 1991PRESENT HEALTH STATUSThe present low level of health status is attributable to lack of political commitment,inappropriate strategies and weakness in implementation of health programmes up to the grassroot level during the past 30 years. Because of those weaknesses even now the crude death rate is16 per thousand, crude birth rate is 41 per thousand, child mortality rate is 107 per thousand,maternal mortality rate is 8.5 per thousand and mortality rate of children below5 years is 197 perthousand. These facts and figures have confirmed Nepal as an underdeveloped and backwardnation.Regarding health services delivery, there is one hospital for 168 thousand persons and onedoctor for 92 thousand persons in the rural areas. Likewise, only one hospital bed is available fornearly 4 thousand persons. There is only one Health post for 24 thousand rural persons, whichindicates the inadequacy of PHC services at the rural level.MAIN DEFICIENCIES IN PREVIOUS HEALTH SERVIC

## Summarization test on sample articles

In [17]:
model = T5Model()
model.to(Config.DEVICE)
model.load_state_dict(
    torch.load(
        Config.MODEL_PATH,
        map_location=torch.device(Config.DEVICE)
        )
    )


<All keys matched successfully>

In [18]:
def get_important_paragraph(article_text):
  data = Process(article_text)
  input_ids, attention_mask = data.pre_process()
  input_ids = input_ids.to(Config.DEVICE)
  attention_mask = attention_mask.to(Config.DEVICE)

  with torch.no_grad():
    generated_ids = model.model.generate(
        input_ids = input_ids,
        attention_mask = attention_mask,
        max_length=150,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

  predicted_high = data.post_process(generated_ids)
  return predicted_high

In [19]:
article_text = "Victims reportedly bled to death when glass-coated strings were entangled around their necks Six people, including three children, have died after their throats were cut by glass-coated kite strings during an annual kite-flying festival in India. Hundreds flocked to terraces and rooftops to unfurl their kites towards the sky at the Uttarayan festival in the western Indian state of Gujarat over the weekend. The victims reportedly bled to death when the razor-sharp kite strings became entangled around their necks, officials told the Press Trust of India (PTI). At least another 176 people were injured due to cuts and falls. Some participants are known to coat their kite strings with powdered glass in order to slice their opponent’s cords while in the air. Although the practice has been banned since 2016, critics say it is rarely enforced. A two-year-old girl reportedly died after a kite string struck her neck while riding on a scooter with her father, PTI cited an official from Bortalav police station as saying. The girl, known only as Kirti, is said to have died during treatment at a hospital on Sunday. Another girl, aged three, was walking home with her mother in Visnagar town on Saturday when a string cut her neck. She was taken to hospital but was declared dead on arrival, a Visnagar police official said. Rishabh Verma, a seven-year-old boy riding on a scooter with his parents, was also struck by a string in Rajkot, an official from Aji Dam police station said. 🔸અમદાવાદમાં ઉત્તરાયણને ધ્યાનમાં રાખી ચુસ્ત પોલીસ બંદોબસ્ત ગોઠવવામાં આવ્યો છે🔸6,000 પોલીસ અધિકારી, કર્મચારીઓ અને 4,000 હોમગાર્ડના જવાનો ખડેપગે રહેશે🔸ચાઈનીઝ દોરી અંગે લોકોને જાગૃત કરવા પોલીસે 175 જેટલી જાગૃતિ સભા પણ યોજી @AhmedabadPolice @sanghaviharsh #Uttrayan #Gujarat pic.twitter.com/adVHjenAyH According to police, similar incidents were reported in Vadodara, Kutch and Gandhinagar districts, where three men were killed in similar circumstances. Nikunj Sharma, a campaigner for People for the Ethical Treatment of Animals (Peta) India, previously told the Guardian the use of manja, the nylon string used to fly kites instead of cotton, could make injuries severe. “Unlike cotton strings these are non-biodegradable, don’t break easily and continue to cause injuries months after the festival,” Sharma said. In Ahmedabad, Gujarat’s largest city, the police department warned against using the coated kites at the festival. “Do not use deadly Chinese manja to fly kites,” it said. Police urged participants to be careful while flying kites during the festival, saying the “momentary fun” of cutting another kite could come at the cost of someone’s life. However, Jayesh Shinde, an activist campaigning against the use of manja, said the government had failed to uphold the ban and should compensate victims for their treatment. “Citizens should not have to pay for the lackadaisical attitude of the government machinery. If the official ban is not being implemented, then the government should offer compensation to victims,” the Times of India quoted Shinde as saying. The popular festival marks the onset of spring where kite-flying is often used during celebrations across the country. Hundreds of birds also fly into the kite strings or are entangled in them, causing deep cuts to their wings, nerve injuries, fractures, dislocations and, in many cases, death."
     
important_paragh = get_important_paragraph(article_text)
important_paragh

'six people, including three children, have died after their throats were cut by glass-coated kite strings during an annual kite-flying festival in India. Hundreds flocked to terraces and rooftops to unfurl their kites towards the sky at the Uttarayan festival in the western Indian state of Gujarat over the weekend. Hundreds flocked to terraces and rooftops to unfurl their kites towards the sky at the Uttarayan festival in the western Indian state of Gujarat over the weekend.'

In [20]:
imp = []
for text in all_text:
    imp.append(get_important_paragraph(text))


In [21]:
imp

['there is one hospital for 168 thousand persons and onedoctor for 92 thousand persons in the rural areas. Likewise, only one hospital bed is available for nearly 4 thousand persons. There is only one Health post for 24 thousand rural persons, which confirms Nepal as an underdeveloped and backwardnation. Because of those weaknesses even now the crude death rate is16 per thousand, crude birth rate is 41 per thousand, maternal mortality rate is 8.5 per thousand and mortality rate of children below5 years is 197 perthousand.',
 "maternal and child health2) Expanded Immunization3) Safe Motherhood4) Diarrhoea and Acute Respiratory Infection Control5) Tuberculosis Control6) Leprosy Control7) Malaria and 'Kalajaar' Control8) Control and Prevention of Communicable Diseases9) Initiation of Prevention of Non-communicable Diseases10) Initiation of Primary Health Services in Urban Slums11) Prevention and Control of AIDS2. PROMOTI VE HEALTH SERVICESThe main programmes under this service are as foll